`time python3 era5_curl.py u10/era5_u10_daily_ v10/era5_v10_daily_ era5_grid.nc era5_lsm.nc era5_curl_daily_19892004.nc`

In [1]:
import sys
import numpy as np
import xarray as xr

/usr/local/lib/python3.6/dist-packages/pandas/compat/_optional.py:106: UserWarning: Pandas requires version '1.2.1' or newer of 'bottleneck' (version '1.2.0' currently installed).
  warnings.warn(msg, UserWarning)


In [2]:
# lat/lon are wrong in the source datasets
def geofix(darray):
    da = xr.DataArray(np.flipud(np.roll(darray.values,720,1)),
                  coords=[darray.lat.values[::-1],np.linspace(-180,179.75,1440)], dims=['lat','lon'])
    return da

In [3]:
def curl_calculate(ds_u, ds_v):
    # Density of air (kg m^-3)
    rho_air = 1.225
    
    # get shape of input arrays
    n_time= len(ds_u.time)
    n_lat = len(ds_u.lat)
    n_lon = len(ds_u.lon)
    
    curl = np.nan*np.zeros([n_time,n_lon,n_lat])
    
    for timeidx in range(0,n_time):
        
        # double zonal_wind_speed(time, lat, lon)
        u = geofix(ds_u[timeidx,:,:])
        v = geofix(ds_v[timeidx,:,:])
        
        # Compute and display wind speed
        ws = np.sqrt(np.square(u)+np.square(v))
        
        # Yelland and Taylor (1996)   
        Cd = (0.29 + 3.1/ws + 7.7/ws)/1000.0
        
        tau_x = rho_air * Cd * (u/np.abs(u))*u**2
        tau_y = rho_air * Cd * (v/np.abs(v))*v**2
        
        ### For each time calculate curl from tau_x and tau_y
        tau_x = tau_x.values.transpose()
        tau_y = tau_y.values.transpose()
        
        for i in range(0,n_lon): #lon
            for j in range(0,n_lat): #lat
                dx = width[i,j]
                dy = height[i,j]
                
                _curl = 0
                _curl += 4./3*(tau_y[i+1 if i+1<n_lon else i+1-n_lon,j] - tau_y[i-1 if i-1>=0 else n_lon-1-i,j])/(2*dx)
                _curl -= 1./6*(tau_y[i+2 if i+2<n_lon else i+2-n_lon,j] - tau_y[i-2 if i-2>=0 else n_lon-1-i,j])/(2*dx)
                _curl -= 4./3*(tau_x[i,j+1 if j+1<n_lat else j+1-n_lat] - tau_x[i,j-1 if j-1>=0 else n_lat-1-j])/(2*dy)
                _curl += 1./6*(tau_x[i,j+2 if j+2<n_lat else j+2-n_lat] - tau_x[i,j-2 if j-2>=0 else n_lat-1-j])/(2*dy)
                curl[timeidx,i,j] = _curl
        
    return curl

In [4]:
# Open grid data
era5_grid = xr.open_dataset('/mnt/efs/data/era5/era5_grid.nc')

In [5]:
width = era5_grid.width.values.transpose()
height = era5_grid.height.values.transpose()

In [17]:
# Open land mask
# lsm(time, latitude, longitude)
era5_lsm = xr.open_dataset('/mnt/efs/data/era5/era5_lsm.nc').rename({'latitude': 'lat', 'longitude': 'lon'})

In [18]:
mask = geofix(era5_lsm.lsm[0,:,:])

In [23]:
# Open wind velocity fields
era5_u10 = xr.open_mfdataset('/mnt/efs/data/era5/u10/era5_u10_daily_*.nc',
                             concat_dim='time').rename({'latitude': 'lat', 'longitude': 'lon'})
era5_v10 = xr.open_mfdataset('/mnt/efs/data/era5/v10/era5_v10_daily_*.nc',
                             concat_dim='time').rename({'latitude': 'lat', 'longitude': 'lon'})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: In xarray version 0.15 the default behaviour of `open_mfdataset`
will change. To retain the existing behavior, pass
combine='nested'. To use future default behavior, pass
combine='by_coords'. See
http://xarray.pydata.org/en/stable/combining.html#combining-multi

  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.6/dist-packages/xarray/backends/api.py:941: FutureWarning: Also `open_mfdataset` will no longer accept a `concat_dim` argument.
To get equivalent behaviour from now on please use the new
`combine_nested` function instead (or the `combine='nested'` option to
`open_mfdataset`).The datasets supplied have global dimension coordinates. You may want
to use the new `combine_by_coords` function (or the
`combine='by_coords'` option to `open_mfdataset`) to order the datasets
before concatenation. Alternatively, to continue concatenating based
on the order t

In [24]:
era5_u10

<xarray.Dataset>
Dimensions:  (lat: 721, lon: 1440, time: 5844)
Coordinates:
  * lat      (lat) float32 90.0 89.75 89.5 89.25 ... -89.25 -89.5 -89.75 -90.0
  * lon      (lon) float32 0.0 0.25 0.5 0.75 1.0 ... 359.0 359.25 359.5 359.75
  * time     (time) datetime64[ns] 1989-01-01 1989-01-02 ... 2004-12-31
Data variables:
    u10      (time, lat, lon) float32 dask.array<chunksize=(365, 721, 1440), meta=np.ndarray>
Attributes:
    Conventions:  CF-1.6
    history:      2020-05-13 02:51:39 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...

In [25]:
times= era5_u10.time.values
lats = era5_grid.lat.values
lons = era5_grid.lon.values

In [26]:
curl = curl_calculate(era5_u10.u10, era5_v10.v10)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:37: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: RuntimeWarning: invalid value encountered in double_scalars


In [31]:
np.shape(curl)

(5844, 1440, 721)

In [28]:
# Build target dataset
era5_curl = xr.Dataset({'curl': ( ['time','lat', 'lon'], curl.transpose([0,2,1]) ),
                        'lsm':  ( ['lat', 'lon'], mask.values )},
                       coords={'time': times, 'lat': lats, 'lon': lons})

In [29]:
# Add attributes and save as NetCDF file
era5_curl['curl'].attrs = {'long_name': 'Wind stress curl', 'units': 'pascals per meter'}
era5_curl['lsm'].attrs  = {'long_name': 'Land-sea mask',    'units': '(0 - 1)'}

In [36]:
era5_curl.to_netcdf('/mnt/efs/data/era5/era5_curl_daily.nc', format='NETCDF4',
                    encoding={'curl': {'zlib': True},
                              'lsm': {'dtype': np.int32, '_FillValue': -32767, 'zlib': True}})

MemoryError: Unable to allocate 16.9 GiB for an array with shape (2191, 721, 1440) and data type float64

In [35]:
era5_curl8994 = era5_curl.sel(time=slice('1989-01-01','1994-12-31'))
era5_curl9504 = era5_curl.sel(time=slice('1995-01-01','2004-12-31'))

In [ ]:
era5_curl8994.to_netcdf('/mnt/efs/data/era5/era5_curl_daily_1989to1994.nc', format='NETCDF4',
                        encoding={'curl': {'zlib': True},
                                  'lsm': {'dtype': np.int32, '_FillValue': -32767, 'zlib': True}})